# Chain of Responsibility Pattern

The Chain of Responsibility pattern passes a request along a chain of handlers. Upon receiving a request, each handler decides either to process the request or to pass it to the next handler in the chain.

## Problem

You need to process a request through multiple handlers, but you don't want to hardcode the sequence or make the client aware of the chain's structure.

## Solution

The Chain of Responsibility pattern suggests organizing handlers into a chain where each handler has a reference to the next handler. The request travels down the chain until a handler processes it or it reaches the end of the chain.

## Simple Example

Here's a straightforward implementation to illustrate the core concepts of the Chain of Responsibility pattern:

In [22]:
class Handler:
    def __init__(self, successor=None):
        self.successor = successor  # Next handler in the chain

    def handle_request(self, request):
        if self.successor:
            return self.successor.handle_request(request)
        return None


class ConcreteHandlerA(Handler):
    def handle_request(self, request):
        if request == "A":
            return "Handled by Handler A"
        return super().handle_request(request)


class ConcreteHandlerB(Handler):
    def handle_request(self, request):
        if request == "B":
            return "Handled by Handler B"
        return super().handle_request(request)


# Setting up the chain
handler_chain = ConcreteHandlerA(ConcreteHandlerB())

# Testing the chain
print(handler_chain.handle_request("A"))  # Output: Handled by Handler A
print(handler_chain.handle_request("B"))  # Output: Handled by Handler B
print(handler_chain.handle_request("C"))  # Output: None

Handled by Handler A
Handled by Handler B
None


In this simple example:

1. We define a base `Handler` class with a successor reference and a `handle_request` method
2. Concrete handlers check if they can process the request; if not, they pass it to the next handler
3. The client creates a chain by linking handlers and sends requests to the first handler

## Real-World Example: Support Ticket System

Let's implement a support ticket system where tickets are handled by different support levels based on priority:

In [23]:
class SupportTicket:
    def __init__(self, id, customer, issue, priority):
        self.id = id
        self.customer = customer
        self.issue = issue
        self.priority = priority  # 1 (low) to 3 (high)

    def __str__(self):
        return f"Ticket #{self.id}: {self.issue} ({self.customer}) - Priority: {self.priority}"

In [24]:
class SupportHandler:
    def __init__(self, name, successor=None):
        self.name = name
        self.successor = successor

    def handle_ticket(self, ticket):
        if self.successor:
            return self.successor.handle_ticket(ticket)
        return f"Ticket #{ticket.id} could not be handled by any support level"


class FirstLevelSupport(SupportHandler):
    def handle_ticket(self, ticket):
        if ticket.priority == 1:
            return f"{self.name} handled ticket #{ticket.id}: {ticket.issue}"
        print(f"{self.name}: Escalating ticket #{ticket.id} (priority {ticket.priority})")
        return super().handle_ticket(ticket)


class SecondLevelSupport(SupportHandler):
    def handle_ticket(self, ticket):
        if ticket.priority <= 2:
            return f"{self.name} handled ticket #{ticket.id}: {ticket.issue}"
        print(f"{self.name}: Escalating ticket #{ticket.id} (priority {ticket.priority})")
        return super().handle_ticket(ticket)


class ProductSpecialist(SupportHandler):
    def handle_ticket(self, ticket):
        return f"{self.name} handled ticket #{ticket.id}: {ticket.issue}"

In [25]:
# Create the support chain
support_chain = FirstLevelSupport(
    "Help Desk", SecondLevelSupport("Technical Support", ProductSpecialist("Product Specialist"))
)

# Create some support tickets
tickets = [
    SupportTicket(1, "John Doe", "Password reset", 1),
    SupportTicket(2, "Jane Smith", "Software installation issue", 2),
    SupportTicket(3, "Bob Johnson", "Server down", 3),
]

# Process tickets through the support chain
for ticket in tickets:
    print(f"\nProcessing: {ticket}")
    result = support_chain.handle_ticket(ticket)
    print(f"Result: {result}")


Processing: Ticket #1: Password reset (John Doe) - Priority: 1
Result: Help Desk handled ticket #1: Password reset

Processing: Ticket #2: Software installation issue (Jane Smith) - Priority: 2
Help Desk: Escalating ticket #2 (priority 2)
Result: Technical Support handled ticket #2: Software installation issue

Processing: Ticket #3: Server down (Bob Johnson) - Priority: 3
Help Desk: Escalating ticket #3 (priority 3)
Technical Support: Escalating ticket #3 (priority 3)
Result: Product Specialist handled ticket #3: Server down


## Benefits of the Chain of Responsibility Pattern

* **Decoupling**: Separates the sender of a request from its receivers
* **Flexibility**: Makes it easy to add new handlers or change the order of handlers
* **Single Responsibility**: Each handler focuses only on its specific responsibility
* **Dynamic Construction**: Chains can be built dynamically at runtime based on specific needs
* **Reduced Conditional Complexity**: Eliminates complex conditional statements by distributing decisions across handlers

## When to Use

* When multiple objects may handle a request, but the handler isn't known in advance
* When you want to issue a request to one of several objects without specifying the receiver explicitly
* When the set of objects that can handle a request should be specified dynamically